### ML with Pyspark
+ classify/predict winning team 

### Data Source
+ https://www.kaggle.com/hikne707/big-five-european-soccer-leagues 

In [2]:
# Load our Packages
from pyspark import SparkContext

In [6]:
sc = SparkContext(master='local[2]')

In [7]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [9]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName("Spark_Spain").getOrCreate()

### workflow
* Data Prep
+ Feature Engineering
+ Build Model 
+ Evaluate 

# Task 
+ predict what team will win in the spain league/country based on parameters
+ multi-classification problem win, lose, or tie

In [21]:
# load our dataset
df = spark.read.csv("D:/Senior/Capstone/data-science-enviroment/data/Leagues/Spain_league_V1.csv", header=True, inferSchema=True)

In [18]:
# data prieview
df.show()

+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+-------+
|Round|                Date|Team_1|Team_2|Year|Country|FT_Team_1|FT_Team_2|HT_Team_1|HT_Team_2|GGD|Team_1_(pts)|Team_2_(pts)|Outcome|
+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+-------+
|    1|(Sat) 2 Sep 1995 ...|   156|   210|1995|      1|        3|        0|        2|        0|  3|           3|           0|      1|
|    1|(Sat) 2 Sep 1995 ...|   187|    21|1995|      1|        3|        0|        3|        0|  3|           3|           0|      1|
|    1|(Sun) 3 Sep 1995 ...|    30|   164|1995|      1|        4|        0|        2|        0|  4|           3|           0|      1|
|    1|(Sun) 3 Sep 1995 ...|    31|   165|1995|      1|        4|        1|        1|        1|  3|           3|           0|      1|
|    1|(Sun) 3 Sep 1995 ...|   152|   172|1995|      1|       

In [16]:
print(df.columns)

['Round', 'Date', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'HT_Team_1', 'HT_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'Outcome']


In [22]:
# check datatypes
# inferschema changes type string to int, creates less work
df.dtypes

[('Round', 'int'),
 ('Date', 'string'),
 ('Team_1', 'int'),
 ('Team_2', 'int'),
 ('Year', 'int'),
 ('Country', 'int'),
 ('FT_Team_1', 'int'),
 ('FT_Team_2', 'int'),
 ('HT_Team_1', 'int'),
 ('HT_Team_2', 'int'),
 ('GGD', 'int'),
 ('Team_1_(pts)', 'int'),
 ('Team_2_(pts)', 'int'),
 ('Outcome', 'int')]

In [23]:
#descriptive summary
print(df.describe().show())

+-------+------------------+--------------------+------------------+------------------+------------------+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|             Round|                Date|            Team_1|            Team_2|              Year|Country|         FT_Team_1|         FT_Team_2|         HT_Team_1|          HT_Team_2|               GGD|      Team_1_(pts)|      Team_2_(pts)|           Outcome|
+-------+------------------+--------------------+------------------+------------------+------------------+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+
|  count|              9554|                9554|              9554|              9554|              9554|   9554|              9554|              9554|              9554|               9554|     

In [25]:
# Value Count (check if dataSet is balance)
df.groupBy('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1| 4569|
|      3| 2422|
|      2| 2563|
+-------+-----+



### Feature Engineering
* Numericcal values
* Vectorization (the process of converting an algorithm from operating on a single value at a time to operating on a set of values (vector) at one time.)
* Scaling

In [26]:
import pyspark.ml
# load Ml packages
from pyspark.ml.feature import VectorAssembler, StringIndexer


In [ ]:
# 2nd half goals full time - half time
df= df.withColumn("H2_Team_1", df['FT_Team_1']-df['HT_Team_1'])
df= df.withColumn("H2_Team_2", df['FT_Team_2']-df['HT_Team_2'])

In [58]:
#Rename HT to represent 1/2 halves, sorry for the redundency spark beginner
df= df.withColumnRenamed('HT_Team_1','H1_Team_1')
df= df.withColumnRenamed('HT_Team_2','H1_Team_2')

In [61]:
#Goal difference given, creating + for home win and - for away win
df= df.withColumn('FT_GD', df['FT_Team_1']-df['FT_Team_2'])
df= df.withColumn('H1_GD', df['H1_Team_1']-df['H1_Team_2'])
df= df.withColumn('H2_GD', df['FT_GD']-df['H1_GD'])



In [62]:
# addded colums include: 
#second half goal difference for home and away
#Full time goal differnce (displays negative if away team scores more)
# goals sccored second half 
# I thought about stripping the date in this case but i dont find it nesscary since we already have the year to work with, date will get dropped later on.
print(df.columns)

['Round', 'Date', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'H1_Team_1', 'H1_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'Outcome', 'H2_Team_1', 'H2_Team_2', 'FT_GD', 'H1_GD', 'H2_GD']


In [63]:
#rearranging column so the output = outcome is at the end
df = df.select('Round', 'Date', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'H1_Team_1', 'H1_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'H2_Team_1', 'H2_Team_2', 'FT_GD', 'H1_GD', 'H2_GD', 'Outcome')

In [64]:
df.show(5)

+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+
|Round|                Date|Team_1|Team_2|Year|Country|FT_Team_1|FT_Team_2|H1_Team_1|H1_Team_2|GGD|Team_1_(pts)|Team_2_(pts)|H2_Team_1|H2_Team_2|FT_GD|H1_GD|H2_GD|Outcome|
+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+
|    1|(Sat) 2 Sep 1995 ...|   156|   210|1995|      1|        3|        0|        2|        0|  3|           3|           0|        1|        0|    3|    2|    1|      1|
|    1|(Sat) 2 Sep 1995 ...|   187|    21|1995|      1|        3|        0|        3|        0|  3|           3|           0|        0|        0|    3|    3|    0|      1|
|    1|(Sun) 3 Sep 1995 ...|    30|   164|1995|      1|        4|        0|        2|        0|  4|           3|           0|        2|     

As you can see home and away team have been label enconded already and converted into numbers(this data was worked on before bringinto jupyter notebook, it is not the raw file, all that is left to do is the date column and as I mention above the initial plan was to drop it but month and year(which we already) can be useful.

In [79]:
# Split the date column to get month, then label encode month
from pyspark.sql.functions import split
df = df.withColumn('Month', split(df['Date'],' ').getItem(2))

In [80]:
# convert the string Month into numbers
# label encoding
monthEncoder = StringIndexer(inputCol='Month',outputCol='Game_Month').fit(df)

In [81]:
df = monthEncoder.transform(df)

In [82]:
df.show(5)

+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+
|Round|                Date|Team_1|Team_2|Year|Country|FT_Team_1|FT_Team_2|H1_Team_1|H1_Team_2|GGD|Team_1_(pts)|Team_2_(pts)|H2_Team_1|H2_Team_2|FT_GD|H1_GD|H2_GD|Outcome|Month|Game_Month|
+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+
|    1|(Sat) 2 Sep 1995 ...|   156|   210|1995|      1|        3|        0|        2|        0|  3|           3|           0|        1|        0|    3|    2|    1|      1|  Sep|       5.0|
|    1|(Sat) 2 Sep 1995 ...|   187|    21|1995|      1|        3|        0|        3|        0|  3|           3|           0|        0|        0|    3|    3|    0|      1|  Sep|       5.0|
|    1|(Sun) 3 Sep 1995 ...|    30|   164|1995|      1|

In [83]:
#get encoded labels, annoying since labels are not from 0-11 jan-dec
monthEncoder.labels

['Apr', 'Jan', 'Mar', 'Feb', 'Oct', 'Sep', 'Nov', 'May', 'Dec', 'Aug', 'Jun']

In [84]:
print(df.columns)

['Round', 'Date', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'H1_Team_1', 'H1_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'H2_Team_1', 'H2_Team_2', 'FT_GD', 'H1_GD', 'H2_GD', 'Outcome', 'Month', 'Game_Month']


In [85]:
required_features = ['Round', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'H1_Team_1', 'H1_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'H2_Team_1', 'H2_Team_2', 'FT_GD', 'H1_GD', 'H2_GD', 'Game_Month', 'Outcome']

In [101]:
# VectorAssembly
vec_assembler = VectorAssembler(inputCols=required_features,outputCol='features')

In [102]:
#sticking entire feature required in a vector which I will be using to Model 
vec_df = vec_assembler.transform(df)

In [90]:
vec_df.show()

+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+--------------------+
|Round|                Date|Team_1|Team_2|Year|Country|FT_Team_1|FT_Team_2|H1_Team_1|H1_Team_2|GGD|Team_1_(pts)|Team_2_(pts)|H2_Team_1|H2_Team_2|FT_GD|H1_GD|H2_GD|Outcome|Month|Game_Month|            features|
+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+--------------------+
|    1|(Sat) 2 Sep 1995 ...|   156|   210|1995|      1|        3|        0|        2|        0|  3|           3|           0|        1|        0|    3|    2|    1|      1|  Sep|       5.0|[1.0,156.0,210.0,...|
|    1|(Sat) 2 Sep 1995 ...|   187|    21|1995|      1|        3|        0|        3|        0|  3|           3|           0|        0|        0|    3|    3|   

### Train, Test, Split

In [103]:
train_df,test_df = vec_df.randomSplit([0.7,0.3])

In [95]:
train_df.count()

6648

### Model Building
+ Pyspark.ml: Dataframe


In [104]:
from pyspark.ml.classification import LogisticRegression 

In [105]:
# logistic model
lr = LogisticRegression(featuresCol='features',labelCol='Outcome')

In [106]:
lr_model = lr.fit(train_df)

In [107]:
y_pred = lr_model.transform(test_df)

In [108]:
y_pred.show()

+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+--------------------+--------------------+--------------------+----------+
|Round|                Date|Team_1|Team_2|Year|Country|FT_Team_1|FT_Team_2|H1_Team_1|H1_Team_2|GGD|Team_1_(pts)|Team_2_(pts)|H2_Team_1|H2_Team_2|FT_GD|H1_GD|H2_GD|Outcome|Month|Game_Month|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+------+----+-------+---------+---------+---------+---------+---+------------+------------+---------+---------+-----+-----+-----+-------+-----+----------+--------------------+--------------------+--------------------+----------+
|    1|(Fri) 17 Aug 2018...|   102|   166|2018|      1|        0|        0|        0|        0|  0|           1|           1|        0|        0|    0|    0|    0|      3|  Aug|       9.0|(19,[0,1,2,3,4,10...|[-

In [110]:
print (y_pred.columns)

['Round', 'Date', 'Team_1', 'Team_2', 'Year', 'Country', 'FT_Team_1', 'FT_Team_2', 'H1_Team_1', 'H1_Team_2', 'GGD', 'Team_1_(pts)', 'Team_2_(pts)', 'H2_Team_1', 'H2_Team_2', 'FT_GD', 'H1_GD', 'H2_GD', 'Outcome', 'Month', 'Game_Month', 'features', 'rawPrediction', 'probability', 'prediction']


In [111]:
y_pred.select('Outcome','rawPrediction', 'probability', 'prediction').show()

+-------+--------------------+--------------------+----------+
|Outcome|       rawPrediction|         probability|prediction|
+-------+--------------------+--------------------+----------+
|      3|[-5132.4851158264...|   [0.0,0.0,0.0,1.0]|       3.0|
|      2|[-5132.8649300937...|   [0.0,0.0,1.0,0.0]|       2.0|
|      1|[-5127.7750910015...|[0.0,1.0,0.0,2.61...|       1.0|
|      2|[-5079.5944439100...|   [0.0,0.0,1.0,0.0]|       2.0|
|      1|[-5117.1766533205...|[0.0,1.0,0.0,2.00...|       1.0|
|      2|[-5127.2188066969...|[0.0,0.0,1.0,2.63...|       2.0|
|      3|[-5122.0905955839...|   [0.0,0.0,0.0,1.0]|       3.0|
|      2|[-5102.4070137056...|[0.0,0.0,1.0,1.86...|       2.0|
|      1|[-5119.9522581968...|[0.0,1.0,0.0,2.25...|       1.0|
|      2|[-5134.5413984142...|[0.0,0.0,1.0,4.96...|       2.0|
|      3|[-5136.1447957354...|[0.0,0.0,1.039827...|       3.0|
|      2|[-5117.3093688592...|[0.0,0.0,1.0,1.06...|       2.0|
|      1|[-5117.4781995909...|[0.0,1.0,0.0,1.52...|    

### Model Evaluation

In [113]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [119]:
#check accuracy
multi_eval = MulticlassClassificationEvaluator(labelCol='Outcome',metricName='accuracy')

In [120]:
multi_eval.evaluate(y_pred)

1.0